In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
pip install focal-loss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install -Uqq ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.2 MB/s eta 0:00:00


In [5]:
import sys
# modify "customized_path_to_homework", path of folder in drive, where you uploaded your homework
customized_path_to_homework = "/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR"
sys.path.append(customized_path_to_homework)

In [6]:
from nltk.corpus import stopwords
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
from utils import clean_str, loadWord2Vec
import sys


In [7]:
#if len(sys.argv) != 2:
#        sys.exit("Use: python remove_words.py <dataset>")

dataset = 'mr'

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stop_words)

{'during', 'with', 'the', 'their', 'before', 'hadn', 'was', 'i', 'below', 'again', 'both', 'shan', 'where', "she's", "doesn't", 'between', "should've", 'here', 'wasn', 'no', 'needn', 'am', 'don', 'very', 'there', 'didn', 've', 'what', 'nor', 'off', 'you', 'when', 'than', 'an', 'or', 'did', 'if', 'after', 'each', 'been', 'a', 'hers', 'myself', 'do', "hadn't", 'once', 'this', 'me', 'own', "you'd", 'he', "that'll", 'and', 'any', 'o', 'we', 'which', 'until', 'that', 'most', 'ma', 'herself', "wouldn't", "wasn't", 'all', "couldn't", 'from', "weren't", 'at', "isn't", 'they', 'just', "needn't", 'aren', 'hasn', 'yourselves', 'its', 'not', 'into', 'down', 'had', "mightn't", 'y', 'mustn', 'them', "don't", 'theirs', 'over', 'now', 'yours', 'to', "it's", 'is', 'same', 'yourself', 'too', 'by', 'how', "you'll", 'those', 'be', 'itself', 'because', 's', 'while', 'such', 'on', 'then', 'under', "you're", 'few', 'ain', 'she', 'were', 'have', 'haven', 'above', 'so', 'about', 'against', 're', 'out', 'having

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
doc_content_list = []
f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/corpus/' + dataset + '.txt', 'rb').read().splitlines();
for line in f: 
    doc_content_list.append(line.strip().decode('latin1')) 


In [9]:
 print(len(doc_content_list))
 

1032


In [10]:
word_freq = {}  # to remove rare words

for doc_content in doc_content_list:
    temp = clean_str(doc_content)
    words = temp.split()
    for word in words:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

In [11]:
clean_docs = []
for doc_content in doc_content_list:
    temp = clean_str(doc_content)
    words = temp.split()
    doc_words = []
    for word in words:
        if dataset == 'mr':
            doc_words.append(word)
        elif word not in stop_words and word_freq[word] >= 5:
            doc_words.append(word)

    doc_str = ' '.join(doc_words).strip()
    clean_docs.append(doc_str)

clean_corpus_str = '\n'.join(clean_docs)

f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/corpus/' + dataset + '.clean.txt', 'w')
f.write(clean_corpus_str)
f.close()

In [12]:
min_len = 10000
aver_len = 0
max_len = 0 

f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/corpus/' + dataset + '.clean.txt', 'r')
lines = f.readlines()
print(len(lines))
for line in lines:
    #print(line)
    line = line.strip()
    temp = line.split()
    aver_len = aver_len + len(temp)
    if len(temp) < min_len:
        min_len = len(temp)
    if len(temp) > max_len:
        max_len = len(temp)
f.close()
aver_len = 1.0 * aver_len / len(lines)
print('min_len : ' + str(min_len))
print('max_len : ' + str(max_len))
print('average_len : ' + str(aver_len))

1032
min_len : 1
max_len : 29
average_len : 10.6531007751938


In [13]:
!pip install -Uqq ipdb
import ipdb

In [14]:
#%pdb on

In [15]:
import os
import random
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from utils import *
from math import log
from sklearn import svm
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk

from sklearn.feature_extraction.text import TfidfVectorizer
import sys
from scipy.spatial.distance import cosine

#if len(sys.argv) != 2:
#	sys.exit("Use: python build_graph.py <dataset>")

# build corpus
dataset = 'mr'

In [16]:
word_embeddings_dim = 300
word_vector_map = {}

# shulffing
doc_name_list = []
doc_train_list = []
doc_test_list = []
Doc_label_list=[]


In [17]:
'''
import pandas as pd
import random
n = 5331 #number of records in file
s = 2612 #desired sample size
skip = sorted(random.sample(range(n),n-s))
data = pd.read_csv('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/Positive_5331_Cleaned_ToSplit.csv',skiprows=skip)
'''

"\nimport pandas as pd\nimport random\nn = 5331 #number of records in file\ns = 2612 #desired sample size\nskip = sorted(random.sample(range(n),n-s))\ndata = pd.read_csv('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/Positive_5331_Cleaned_ToSplit.csv',skiprows=skip)\n"

In [18]:
'''
import pandas as pd
from sklearn.model_selection import train_test_split
#data = pd.read_csv('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/Positive_5331_Cleaned_ToSplit.csv')
data = pd.read_csv('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/Majority_minority.csv')
data.info()
train, test = train_test_split(data, random_state = 42, test_size=0.2)
print(train.shape)
print(test.shape)
'''

"\nimport pandas as pd\nfrom sklearn.model_selection import train_test_split\n#data = pd.read_csv('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/Positive_5331_Cleaned_ToSplit.csv')\ndata = pd.read_csv('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/Majority_minority.csv')\ndata.info()\ntrain, test = train_test_split(data, random_state = 42, test_size=0.2)\nprint(train.shape)\nprint(test.shape)\n"

In [19]:
#data 

In [20]:
# Problem 1) iii) Save the dataset as csv file with "export_dataframe" name
#train.to_csv (r'/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/export_train.csv',index = False, header=False)

In [21]:
# Problem 1) iii) Save the dataset as csv file with "export_dataframe" name
#test.to_csv (r'/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/export_test.csv',index = False, header=False)

In [22]:
# train test split
f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/' + dataset + '.txt', 'r') # 3 columns, path, train/test, label
lines = f.readlines()
for line in lines:
    doc_name_list.append(line.strip())
    #temp2 = line.split("\t")
    temp = line.split("\t")
    if temp[1].find('test') != -1:
        doc_test_list.append(line.strip())
        #Doc_label_list.append(temp[2])
    elif temp[1].find('train') != -1:
        doc_train_list.append(line.strip())
        Doc_label_list.append(temp[2])
f.close()


In [23]:
 print((Doc_label_list))

['0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '1\n', '1\n', '0\n', '0\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '1\n', '0\n', '1\n', '1\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '1\n', '0\n', '0\n', '1\n', '0\n', '0\n', '1\n', '0\n', '1\n', '0\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '1\n', '1\n', '1\n', '1\n', '0\n', '0\n', '0\n', '0\n', '0\n', '1\n', '0\n', '1\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '1\n', '1\n', '1\n', '1\n', '1\n', '1\n', '0\n', '0\n', '1\n', '1\n', '0\n', '0\n', '0\n', '1\n', '0\n', '1\n', '0\n', '1\n', '0\n', '1\n', '1\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '0\n', '0\n', '1\n', '1\n', '0\n'

In [24]:
 print(len(doc_test_list))
 print(len(doc_train_list))

205
827


In [25]:
# get an idea of the distribution of the text values
from sklearn.utils import class_weight
from collections import Counter
##ctr = Counter(trainData['class'].values)
#print('Distribution of Classes:', ctr)

# get class weights for the training data, this will be used data
y_train_int = (Doc_label_list)
cws = class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(y_train_int),  y=y_train_int)
print(cws) 
class_weightss = dict(zip(np.unique(y_train_int), class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(y_train_int),  y=y_train_int))) 
print(class_weightss)

[0.75045372 1.49818841]
{'0\n': 0.7504537205081669, '1\n': 1.4981884057971016}


In [26]:
doc_content_list = []
f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/corpus/' + dataset + '.clean.txt', 'r') # clean: after stop/rare words filtering
lines = f.readlines()
for line in lines:
    doc_content_list.append(line.strip())
f.close()

In [27]:
train_ids = []
for train_name in doc_train_list:
    train_id = doc_name_list.index(train_name)
    train_ids.append(train_id)
## print(train_ids)

In [28]:
print(train_ids)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [29]:
#random.shuffle(train_ids)

In [30]:
# partial labeled data, if you only want 20% training set
#train_ids = train_ids[:int(0.2 * len(train_ids))]

# persisting the training set 
train_ids_str = '\n'.join(str(index) for index in train_ids)
f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/' + dataset + '.train.index', 'w')
f.write(train_ids_str)
f.close()

In [31]:
# may not be necessary
test_ids = []
for test_name in doc_test_list:
    test_id = doc_name_list.index(test_name)
    test_ids.append(test_id)
## print(test_ids)
#random.shuffle(test_ids)

test_ids_str = '\n'.join(str(index) for index in test_ids)
f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/' + dataset + '.test.index', 'w')
f.write(test_ids_str)
f.close()

In [32]:
ids = train_ids + test_ids
## print(ids)
print(len(ids))

shuffle_doc_name_list = []
shuffle_doc_words_list = []
for id in ids:
    shuffle_doc_name_list.append(doc_name_list[int(id)])
    shuffle_doc_words_list.append(doc_content_list[int(id)])
shuffle_doc_name_str = '\n'.join(shuffle_doc_name_list)
shuffle_doc_words_str = '\n'.join(shuffle_doc_words_list) # content

1032


In [33]:
print(ids)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [34]:
f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/' + dataset + '_shuffle.txt', 'w')
f.write(shuffle_doc_name_str)
f.close()

f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/corpus/' + dataset + '_shuffle.txt', 'w')
f.write(shuffle_doc_words_str)
f.close()


In [35]:
# build vocab using cleaned words and record freq.
word_freq = {}
word_set = set()
for doc_words in shuffle_doc_words_list:
    words = doc_words.split()
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

word_doc_list = {}

In [36]:
# keep doc occurrence list, for idf
for i in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    appeared = set()
    for word in words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)


In [37]:
## df
word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)
## from word to id
word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i


In [38]:
vocab_str = '\n'.join(vocab)

f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/corpus/' + dataset + '_vocab.txt', 'w')
f.write(vocab_str)
f.close()


In [39]:
# get unique label list
label_set = set()
for doc_meta in shuffle_doc_name_list:
    temp = doc_meta.split('\t')
    label_set.add(temp[2])
label_list = list(label_set)

In [40]:
# x: feature vectors of training docs, no initial features, one hot input
# slect 90% training set
train_size = len(train_ids)
val_size = int(0.1 * train_size)
real_train_size = train_size - val_size 
# different training rates

In [41]:
real_train_doc_names = shuffle_doc_name_list[:real_train_size]
real_train_doc_names_str = '\n'.join(real_train_doc_names)

f = open('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/' + dataset + '.real_train.name', 'w')
f.write(real_train_doc_names_str)
f.close()


In [42]:
## not necessary if don't use preloaded embedding
row_x = []
col_x = []
data_x = []
for i in range(real_train_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec #+ np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_x.append(i)
        col_x.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_x.append(doc_vec[j] / doc_len)

# x = sp.csr_matrix((real_train_size, word_embeddings_dim), dtype=np.float32)
x = sp.csr_matrix((data_x, (row_x, col_x)), shape=(
    real_train_size, word_embeddings_dim))


In [43]:
y = []
for i in range(real_train_size):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    y.append(one_hot)
y = np.array(y)
#print(y)

In [44]:
# tx: feature vectors of test docs, no initial features
test_size = len(test_ids)

row_tx = []
col_tx = []
data_tx = []
for i in range(test_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i + train_size]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec #+ np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_tx.append(i)
        col_tx.append(j)
        data_tx.append(doc_vec[j] / doc_len)  # doc_vec[j] / doc_len

tx = sp.csr_matrix((data_tx, (row_tx, col_tx)),
                   shape=(test_size, word_embeddings_dim))


In [45]:
ty = []
for i in range(test_size):
    doc_meta = shuffle_doc_name_list[i + train_size]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ty.append(one_hot)
ty = np.array(ty)
## print(ty)

In [46]:
#print(ty)

In [47]:
# allx: the the feature vectors of both labeled and unlabeled training instances
# (a superset of x) train+val+word list
# unlabeled training instances -> words

word_vectors = np.random.uniform(-0.01, 0.01,
                                 (vocab_size, word_embeddings_dim))

row_allx = []
col_allx = []
data_allx = []

In [48]:
for i in range(train_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec #+ np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_allx.append(int(i))
        col_allx.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_allx.append(doc_vec[j] / doc_len)  # doc_vec[j]/doc_len

for i in range(vocab_size):
    for j in range(word_embeddings_dim):
        row_allx.append(int(i + train_size))
        col_allx.append(j)
        data_allx.append(word_vectors.item((i, j)))


row_allx = np.array(row_allx)
col_allx = np.array(col_allx)
data_allx = np.array(data_allx)


allx = sp.csr_matrix((data_allx, (row_allx, col_allx)),
shape=(train_size + vocab_size, word_embeddings_dim))



In [49]:
ally = []
for i in range(train_size):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ally.append(one_hot)

In [50]:
## dummy label for vocab, not counted in loss and learning
for i in range(vocab_size):
    one_hot = [0 for l in range(len(label_list))]
    ally.append(one_hot)

ally = np.array(ally)

print(x.shape, y.shape, tx.shape, ty.shape, allx.shape, ally.shape)

(745, 300) (745, 2) (205, 300) (205, 2) (4351, 300) (4351, 2)


In [51]:
'''
Doc word heterogeneous graph
'''

# word co-occurence with context windows
window_size = 20
windows = []

for doc_words in shuffle_doc_words_list:
    words = doc_words.split()
    length = len(words)
    if length <= window_size:
        windows.append(words)
    else:
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            # print(window)


In [52]:
word_window_freq = {} # number of windows a word occurs in
for window in windows:
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])


In [53]:
## number of windows a pair of words occur in
word_pair_count = {}
for window in windows:
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i]
            word_i_id = word_id_map[word_i]
            word_j = window[j]
            word_j_id = word_id_map[word_j]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1

In [54]:
row = []
col = []
weight = []

# pmi as weights

num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    
    row.append(train_size + i)
    col.append(train_size + j)
    weight.append(pmi)

In [55]:
# doc word frequency, tf
doc_word_freq = {}

for doc_id in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[doc_id]
    words = doc_words.split()
    for word in words:
        word_id = word_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1

In [56]:
# tfidf
for i in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_word_set = set()
    for word in words:
        if word in doc_word_set:
            continue
        j = word_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        if i < train_size:
            row.append(i)
        else:
            row.append(i + vocab_size)
        col.append(train_size + j)
        idf = log(1.0 * len(shuffle_doc_words_list) /
                  word_doc_freq[vocab[j]])
        weight.append(freq * idf)
        doc_word_set.add(word)


In [57]:
## adjacent matrix
node_size = train_size + vocab_size + test_size
adj = sp.csr_matrix(
    (weight, (row, col)), shape=(node_size, node_size))

In [58]:
# save objects as input for train.py
f = open("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/ind.{}.x".format(dataset), 'wb')
pkl.dump(x, f)
f.close()

f = open("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/ind.{}.y".format(dataset), 'wb')
pkl.dump(y, f)
f.close()

f = open("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/ind.{}.tx".format(dataset), 'wb')
pkl.dump(tx, f)
f.close()

f = open("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/ind.{}.ty".format(dataset), 'wb')
pkl.dump(ty, f)
f.close()

f = open("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/ind.{}.allx".format(dataset), 'wb')
pkl.dump(allx, f)
f.close()

f = open("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/ind.{}.ally".format(dataset), 'wb')
pkl.dump(ally, f)
f.close()

f = open("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/ind.{}.adj".format(dataset), 'wb')
pkl.dump(adj, f)
f.close()

In [59]:
from __future__ import division
from __future__ import print_function

import time
#import tensorflow as tf

from sklearn import metrics
from utils import *
from models import GCN
import random
import os
import sys
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#if len(sys.argv) != 2:
#	sys.exit("Use: python train.py <dataset>")

dataset = 'mr'

# Set random seed
seed = random.randint(1, 200)
tf.set_random_seed(seed)
#tf.set_random_seed(3)

# Settings, default not using GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

flags = tf.app.flags
FLAGS = flags.FLAGS
# 'cora', 'citeseer', 'pubmed'
flags.DEFINE_string('dataset', dataset, 'Dataset string.')
# 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_string('model', 'gcn', 'Model string.')
flags.DEFINE_float('learning_rate', 0.02, 'Initial learning rate.')
#flags.DEFINE_float('learning_rate', 0.0001, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 200, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 0,
                   'Weight for L2 loss on embedding matrix.')  # 5e-4
flags.DEFINE_integer('early_stopping', 10,
                     'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')



In [60]:
 #FLAGS['learning_rate'].value=0.0001
 #FLAGS['dropout'].value=0.2
 #FLAGS['weight_decay'].value=0
FLAGS['epochs'].value=200

In [61]:
 #Set random seed
#seed = random.randint(1, 200)
#tf.set_random_seed(seed)
#tf.set_random_seed(3)

In [62]:
# Load data

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus(
   FLAGS['dataset'].value)

(745, 300) (745, 2) (205, 300) (205, 2) (4351, 300) (4351, 2)
4556


In [63]:
#print(adj)
features = sp.identity(features.shape[0])  # featureless

In [64]:
print(adj)

  (0, 1198)	1.935307640096049
  (0, 1266)	3.7612001156935624
  (0, 1355)	3.9948149668750674
  (0, 1884)	4.742029368705289
  (0, 1973)	4.636668853047462
  (0, 2497)	6.246106765481563
  (0, 2526)	6.246106765481563
  (0, 3244)	4.1666652238017265
  (0, 3598)	5.840641657373398
  (0, 3812)	5.552959584921617
  (0, 4102)	6.246106765481563
  (0, 4138)	4.454347296253507
  (1, 832)	6.939253946041508
  (1, 965)	5.840641657373398
  (1, 1416)	5.840641657373398
  (1, 3060)	5.840641657373398
  (1, 3068)	6.939253946041508
  (1, 3141)	6.939253946041508
  (1, 3491)	6.246106765481563
  (1, 4286)	5.840641657373398
  (2, 1414)	6.246106765481563
  (2, 1644)	3.720378121173307
  (2, 1736)	6.246106765481563
  (2, 1824)	3.5380565643793527
  (2, 1969)	5.329816033607408
  :	:
  (4554, 1198)	1.935307640096049
  (4554, 1667)	4.993343796986195
  (4554, 1772)	4.1666652238017265
  (4554, 2294)	2.5572273113676265
  (4554, 2801)	4.859812404361672
  (4554, 3071)	6.939253946041508
  (4554, 3225)	4.859812404361672
  (4554, 

In [65]:
print(adj.shape)
print(features.shape)

# Some preprocessing
features = preprocess_features(features)
if FLAGS['model'].value == 'gcn':
    support = [preprocess_adj(adj)]
    num_supports = 1 # list size 1, different from Kipf
else:
    raise ValueError('Invalid argument for model: ' + str(FLAGS['model'].value))


(4556, 4556)
(4556, 4556)


In [66]:
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    # helper variable for sparse dropout
    'num_features_nonzero': tf.placeholder(tf.int32),
}


In [67]:
# Create model
print(features[2][1]) # dim, note there is sparse_to_tuple in preprocess_features
model = GCN(placeholders, input_dim=features[2][1], logging=True)



4556


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [68]:
# Initialize session
session_conf = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=session_conf)


In [69]:
# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(
        features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy, model.pred, model.labels,model.outputs], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], outs_val[2], outs_val[3],outs_val[4], (time.time() - t_test)

In [87]:
#tf.set_random_seed(7.1)
# Init variables, models.py call layers.py then call init.py
sess.run(tf.global_variables_initializer()) 

cost_val = []

# Train model
for epoch in range(FLAGS['epochs'].value):

    t = time.time()
    # Construct feed dictionary, same input for every epoch as we input full batch
    feed_dict = construct_feed_dict(
        features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS['dropout'].value}) # update dropout for train, different from test 0
    # to inductive
    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy,
                     model.layers[0].embedding,model.outputs], feed_dict=feed_dict)

    # Validation
    cost, acc, pred, labels,output_val, duration = evaluate(
        features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(
              outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS['early_stopping'].value and cost_val[-1] > np.mean(cost_val[-(FLAGS['early_stopping'].value+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

Epoch: 0001 train_loss= 0.69419 train_acc= 0.37450 val_loss= 0.68339 val_acc= 0.56098 time= 0.09303
Epoch: 0002 train_loss= 0.68328 train_acc= 0.85369 val_loss= 0.68145 val_acc= 0.57317 time= 0.10222
Epoch: 0003 train_loss= 0.66372 train_acc= 0.90201 val_loss= 0.67820 val_acc= 0.58537 time= 0.11427
Epoch: 0004 train_loss= 0.63561 train_acc= 0.90067 val_loss= 0.67507 val_acc= 0.57317 time= 0.15644
Epoch: 0005 train_loss= 0.60051 train_acc= 0.90336 val_loss= 0.67389 val_acc= 0.57317 time= 0.14885
Epoch: 0006 train_loss= 0.55862 train_acc= 0.92081 val_loss= 0.67677 val_acc= 0.54878 time= 0.14323
Epoch: 0007 train_loss= 0.50247 train_acc= 0.92349 val_loss= 0.68568 val_acc= 0.56098 time= 0.14762
Epoch: 0008 train_loss= 0.46101 train_acc= 0.91275 val_loss= 0.70259 val_acc= 0.56098 time= 0.13481
Epoch: 0009 train_loss= 0.40292 train_acc= 0.91678 val_loss= 0.73100 val_acc= 0.54878 time= 0.14206
Epoch: 0010 train_loss= 0.36450 train_acc= 0.92886 val_loss= 0.77158 val_acc= 0.56098 time= 0.14132


In [71]:
outs[3]

array([[ 0.07051446,  0.06394844,  0.0599598 , ...,  0.07103068,
        -0.00130428,  0.06543893],
       [ 0.07948031,  0.07922494,  0.06945902, ...,  0.0300811 ,
        -0.02408106,  0.07884413],
       [ 0.02300238,  0.01228034, -0.00270218, ...,  0.0067099 ,
         0.07639411,  0.01046613],
       ...,
       [ 0.02568352, -0.00190981,  0.01226411, ...,  0.00643552,
         0.04614057,  0.02359171],
       [ 0.01046518,  0.01089563,  0.00704239, ...,  0.01299682,
         0.01243574,  0.00115368],
       [-0.01237618, -0.02033091, -0.01029628, ..., -0.00854484,
         0.10447925,  0.01742101]], dtype=float32)

In [72]:
outs[4]

array([[ 0.5714621 , -0.5568992 ],
       [ 0.6327216 , -0.653036  ],
       [-0.5456739 ,  0.63360316],
       ...,
       [-0.46073043,  0.5167562 ],
       [ 0.12866782, -0.10336556],
       [-0.33054498,  0.37672013]], dtype=float32)

In [88]:
# Testing
test_cost, test_acc, pred, labels,output_test,test_duration = evaluate(
    features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))

test_pred = []
test_labels = []
print(len(test_mask))
for i in range(len(test_mask)):
    if test_mask[i]:
        test_pred.append(pred[i])
        test_labels.append(labels[i])

print("Test Precision, Recall and F1-Score...")
print(metrics.classification_report(test_labels, test_pred, digits=4))
from sklearn.metrics import roc_curve,roc_auc_score,average_precision_score

fpr , tpr , thresholds = roc_curve ( test_labels ,test_pred)
auc_score=roc_auc_score(test_labels ,test_pred)
print('auc_score',round(auc_score,4))
AVg_Precision=average_precision_score(test_labels ,test_pred)
print('AVg_Precision',round(AVg_Precision,4))
from imblearn.metrics import geometric_mean_score
Gmean=geometric_mean_score(test_labels ,test_pred)
print('GMEAN',round(Gmean,4))
print("**********************************")
print("Macro average Test Precision, Recall and F1-Score...")
print(metrics.precision_recall_fscore_support(test_labels, test_pred, average='macro'))
print("Micro average Test Precision, Recall and F1-Score...")
print(metrics.precision_recall_fscore_support(test_labels, test_pred, average='micro'))
print(label_list)
labelList=["Negative","Positive"]

#confusion_ma(test_labels, test_pred, labelList)


Test set results: cost= 0.73897 accuracy= 0.64878 time= 0.05868
4556
Test Precision, Recall and F1-Score...
              precision    recall  f1-score   support

           0     0.7407    0.7299    0.7353       137
           1     0.4714    0.4853    0.4783        68

    accuracy                         0.6488       205
   macro avg     0.6061    0.6076    0.6068       205
weighted avg     0.6514    0.6488    0.6500       205

auc_score 0.6076
AVg_Precision 0.3995
GMEAN 0.5952
**********************************
Macro average Test Precision, Recall and F1-Score...
(0.606084656084656, 0.6076105624731645, 0.6067774936061381, None)
Micro average Test Precision, Recall and F1-Score...
(0.6487804878048781, 0.6487804878048781, 0.6487804878048781, None)
['0', '1']


In [74]:
output_test

array([[ 0.9284805 , -0.92401874],
       [ 1.2074939 , -1.1858294 ],
       [-0.5557454 ,  0.6033905 ],
       ...,
       [-0.26358846,  0.30282056],
       [ 0.1334039 , -0.10603443],
       [-0.28712007,  0.32621107]], dtype=float32)

In [75]:
np.savetxt("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/GCN_Train/test_pred.txt", test_pred, fmt = "%d")

In [76]:
np.savetxt("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/GCN_Train/test_labels.txt", test_labels, fmt = "%d")

In [77]:
np.savetxt("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/GCN_Train/y_train.txt", y_train, fmt = "%d")

In [78]:
Ola

NameError: ignored

In [ ]:
m1=fpr
m2=tpr
m3=thresholds

In [ ]:
m4=fpr2
m5=tpr2
m6=thresholds2

In [ ]:
m7=fpr3
m8=tpr3
m9=thresholds3

In [ ]:
fpr_TextGCN=[]
tpr_TextGCN=[]
threshold_TextGCN=[]
print(m1)
print(m2)
print(m3)
print(m4)
print(m5)
print(m6)
print(m7)
print(m8)
print(m9)
fpr_TextGCN.append(m1)
fpr_TextGCN.append(m4)
fpr_TextGCN.append(m7)
tpr_TextGCN.append(m2)
tpr_TextGCN.append(m5)
tpr_TextGCN.append(m8)
threshold_TextGCN.append(m3)
threshold_TextGCN.append(m6)
threshold_TextGCN.append(m9)
def avg_CM(ConfusionMatrix):
   sum_e1=0
   for array in ConfusionMatrix:
    #print(array)
    sum_e1+=array
    #print(sum_e1)
   average_CM=sum_e1/len(ConfusionMatrix)
   
       
   return  average_CM  
fpr_TextGCN=avg_CM(fpr_TextGCN) 
tpr_TextGCN=avg_CM(tpr_TextGCN) 
threshold=avg_CM(threshold_TextGCN) 
print(fpr_TextGCN)
print(tpr_TextGCN)
print(threshold_TextGCN)   

In [ ]:
tpr_TextGCN

In [ ]:
np.savetxt("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/GCN_Train/outputLayer.txt", outs[4], fmt = "%f")

In [ ]:
outs[4]

In [ ]:
np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])

In [ ]:
def count_number_trainable_params():
    '''
    Counts the number of trainable variables.
    '''
    tot_nb_params = 0
    for trainable_variable in tf.trainable_variables():
        shape = trainable_variable.get_shape() # e.g [D,F] or [W,H,C]
        print (shape) 
        current_nb_params = get_nb_params_shape(shape)
        print (current_nb_params) 
        tot_nb_params = tot_nb_params + current_nb_params
    return tot_nb_params
def get_nb_params_shape(shape):
    '''
    Computes the total number of params for a given shap.
    Works for any number of shapes etc [D,F] or [W,H,C] computes D*F and W*H*C.
    '''
    nb_params = 1
    for dim in shape:
        nb_params = nb_params*int(dim)
    return nb_params 

In [ ]:
count_number_trainable_params()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
def confusion_ma(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.savefig('/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/Text-GCN.png', dpi=300)
    return plt.show()

In [ ]:
np.savetxt("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/GCN_Validation/output_val.txt", output_val, fmt = "%f")

In [ ]:
np.savetxt("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_MR/data/GCN_Test/output_test.txt", output_test, fmt = "%f")

In [ ]:
outs[4]

In [ ]:
print(label_list)

In [ ]:
np.savetxt("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_DLRL/data/train_mask.txt", train_mask, fmt = "%s")

In [ ]:
np.savetxt("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_DLRL/data/test_mask.txt", test_mask, fmt = "%s")

In [ ]:
np.savetxt("/content/drive/MyDrive/GCN/Graph_OneClass/Contribution/text_gcn_DLRL/data/val_mask.txt", val_mask, fmt = "%s")